In [2]:
from endless_line.data_utils.dashboard_utils import DashboardUtils
import pandas as pd
from datetime import timedelta, datetime

dashboard_utils = DashboardUtils()

In [3]:
current_date = datetime.strptime('2025-02-13', '%Y-%m-%d')
start_date = current_date - timedelta(days=5)
end_date = datetime.strptime('2025-02-16', '%Y-%m-%d')

hist, pred = dashboard_utils.get_predicted_attendance_with_past(start_date, current_date, end_date)



/Users/hadrienmorand/Documents/Studies/ESSEC_CENTRALE/Studies/M2/T2/Centrale/Hackathon/Hackathon-ElevenStrategy/endless_line/data_utils/weather_forecast.py:73: FutureWarning: DataFrame.interpolate with method=ffill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  forecast_df = forecast_df.set_index('dt_iso').resample('h').interpolate(method='ffill', limit_direction='forward')


In [4]:
import plotly.graph_objects as go

# Ensure data is sorted
df = dashboard_utils.data.attendance

# Manually add transition point to keep continuity
transition_x = [hist["USAGE_DATE"].iloc[-1], pred["USAGE_DATE"].iloc[0]]
transition_y = [hist["attendance"].iloc[-1], pred["attendance"].iloc[0]]

fig = go.Figure()

# Historical data (Blue Line)
fig.add_trace(go.Scatter(
    x=hist["USAGE_DATE"],
    y=hist["attendance"],
    mode="lines",
    line=dict(color="blue", width=2),
    line_shape="spline",
    name="Historical Attendance"
))

# Transition Line (Connect Last Historical to First Prediction)
fig.add_trace(go.Scatter(
    x=transition_x,
    y=transition_y,
    mode="lines",
    line=dict(color="blue", width=2, dash="dot"),
    line_shape="spline",
    showlegend=False
))

# Predicted Data (Red Line)
fig.add_trace(go.Scatter(
    x=pred["USAGE_DATE"],
    y=pred["attendance"],
    mode="lines",
    line=dict(color="red", width=2),
    line_shape="spline",
    name="Predicted Attendance"
))

# Layout enhancements
fig.update_layout(
    title="Attendance Prediction",
    xaxis_title="Date",
    yaxis_title="Attendance",
    hovermode="x unified",
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    plot_bgcolor="white"
)

fig.show()


In [11]:
attractions = list(data.link_attraction_park.ATTRACTION.unique())

In [12]:
attractions

['Bumper Cars',
 'Bungee Jump',
 'Circus Train',
 'Crazy Dance',
 'Dizzy Dropper',
 'Drop Tower',
 'Flying Coaster',
 'Free Fall',
 'Giant Wheel',
 'Giga Coaster',
 'Go-Karts',
 'Haunted House',
 'Himalaya Ride',
 'Inverted Coaster',
 'Kiddie Coaster',
 'Merry Go Round',
 'Oz Theatre',
 'Rapids Ride',
 'Roller Coaster',
 'Spinning Coaster',
 'Spiral Slide',
 'Superman Ride',
 'Swing Ride',
 'Vertical Drop',
 'Water Ride',
 'Zipline']

In [57]:
import boto3
from botocore.config import Config
from botocore.exceptions import ClientError
from dotenv import load_dotenv
import os
from endless_line.data_utils.dataloader import DataLoader

data = DataLoader()

load_dotenv(os.path.join(data.root_dir, '.secret'))


key_id = os.getenv('B2keyID')
db_name = os.getenv('B2DBNAME')
key_name = os.getenv('B2keyNAME')
key_app_key = os.getenv('B2keyAPPKEY')
endpoint = os.getenv('B2endpoint')

def get_b2_resource(key_id, key_app_key, url_endpoint):
    config = Config(
        signature_version='s3v4')
    return boto3.resource(service_name='s3',
                          config=config,
                          endpoint_url=url_endpoint,
                          aws_access_key_id=key_id,
                          aws_secret_access_key=key_app_key)

resource = get_b2_resource(key_id, key_app_key, endpoint)

bucket = resource.Bucket(db_name)

In [60]:
'weather_data.csv' in [obj.key for obj in bucket.objects.all()]

True

In [58]:
csv_files = [obj.key for obj in resource.Bucket(db_name).objects.all() if obj.key.endswith('.csv')]

dfs = {}  # Dictionary to store DataFrames
print(csv_files)
for csv_key in csv_files:
    obj = resource.Object(db_name, csv_key)
    csv_data = obj.get()['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_data))
    dfs[csv_key] = df  # Store DataFrame with filename as key
    print(f"Loaded {csv_key}")

# Example: Print first few rows of a specific file
dfs["attendance.csv"].head()

['attendance.csv', 'entity_schedule.csv', 'link_attraction_park.csv', 'waiting_times.csv', 'weather_data.csv']
Loaded attendance.csv
Loaded entity_schedule.csv
Loaded link_attraction_park.csv


KeyboardInterrupt: 

In [2]:
from endless_line.data_utils.dashboard_utils import DashboardUtils


board = DashboardUtils()
board.get_attractions()

['Bumper Cars',
 'Bungee Jump',
 'Circus Train',
 'Crazy Dance',
 'Dizzy Dropper',
 'Drop Tower',
 'Flying Coaster',
 'Free Fall',
 'Giant Wheel',
 'Giga Coaster',
 'Go-Karts',
 'Haunted House',
 'Himalaya Ride',
 'Inverted Coaster',
 'Kiddie Coaster',
 'Merry Go Round',
 'Oz Theatre',
 'Rapids Ride',
 'Roller Coaster',
 'Spinning Coaster',
 'Spiral Slide',
 'Superman Ride',
 'Swing Ride',
 'Vertical Drop',
 'Water Ride',
 'Zipline']

In [35]:
data = DataLoader(load_all_files=True)
data.clean_data()
waiting_time = data.waiting_times
waiting_time.head()

/Users/theovaneccelpoel/Desktop/Hackathon/Hackathon-ElevenStrategy/endless_line/data_utils/dataloader.py:253: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.parade_night_show_attendance["Num_parade"] = 3 - self.parade_night_show_attendance[["NIGHT_SHOW",	"PARADE_1",	"PARADE_2"]].isna().sum(axis=1)


,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT
0,2018-01-01,2018-01-01 21:00:00,21,2018-01-01 21:15:00,Roller Coaster,0,2.0,0.0,0.000,0.00,0,0,0,2.0
1,2018-01-01,2018-01-01 19:30:00,19,2018-01-01 19:45:00,Bumper Cars,5,18.0,148.0,254.749,254.75,15,15,0,18.0
2,2018-01-01,2018-01-01 22:30:00,22,2018-01-01 22:45:00,Rapids Ride,0,1.0,0.0,0.000,0.00,0,0,0,2.0
3,2018-01-01,2018-01-01 12:45:00,12,2018-01-01 13:00:00,Crazy Dance,5,1.0,46.0,250.001,250.00,15,15,0,1.0
5,2018-01-01,2018-01-01 18:15:00,18,2018-01-01 18:30:00,Free Fall,50,3.0,0.0,0.000,0.00,0,0,0,3.0


20.557254464285716